In [1]:
import os
import pandas as pd

In [2]:
# Root directory of your project
ROOT_DIR = "../" 

RAW_DATA_DIR = os.path.join(ROOT_DIR, "data", "raw")

In [3]:
quarters = ["2024q1", "2024q2", "2024q3", "2024q4", "2025q1", "2025q2"]
all_annual_filings = []

In [4]:
# print(sub_df.columns)


In [5]:
for q in quarters:
    folder = os.path.join(RAW_DATA_DIR, q)
    sub_path = os.path.join(folder, "sub.txt")
    
    # 1) Load sub.txt
    sub_df = pd.read_csv(sub_path, sep="\t", low_memory=False)
    
    # 2) Filter only 10-K filings (annual filings)
    annual_filings = sub_df.loc[sub_df["form"] == "10-K"].copy()
    
    annual_filings["quarter"] = q
    all_annual_filings.append(annual_filings)

In [6]:
# Combine results across quarters
annual_filings_all = pd.concat(all_annual_filings, ignore_index=True)

print(f"Total number of 10-K filings found: {len(annual_filings_all)}")
annual_filings_all[["cik","name","adsh","filed","form","quarter"]].head()

Total number of 10-K filings found: 10578


,cik,name,adsh,filed,form,quarter
0,1059556,MOODYS CORP /DE/,0001059556-24-000017,20240214,10-K,2024q1
1,1206264,"TEMPUR SEALY INTERNATIONAL, INC.",0001206264-24-000050,20240216,10-K,2024q1
2,1171759,RED ROBIN GOURMET BURGERS INC,0001171759-24-000011,20240228,10-K,2024q1
3,891532,PERMA FIX ENVIRONMENTAL SERVICES INC,0001493152-24-009805,20240313,10-K,2024q1
4,885725,BOSTON SCIENTIFIC CORP,0000885725-24-000017,20240220,10-K,2024q1


In [7]:
filtered_num_all = []

In [8]:
for q in quarters:
    folder = os.path.join(RAW_DATA_DIR, q)
    num_path = os.path.join(folder, "num.txt")
    
    # Load num.txt   (also TAB-separated)
    num_df = pd.read_csv(num_path, sep="\t", low_memory=False)
    
    # Get the list of adsh values for the 10-K filings in this quarter
    adsh_q = annual_filings_all.loc[annual_filings_all["quarter"] == q, "adsh"].tolist()
    
    # Filter num_df
    num_filtered = num_df[num_df["adsh"].isin(adsh_q)].copy()
    num_filtered["quarter"] = q
    
    filtered_num_all.append(num_filtered)

In [9]:
# Combine all quarters into one dataframe
filtered_num_all_df = pd.concat(filtered_num_all, ignore_index=True)

print(f"Total number of financial facts extracted: {len(filtered_num_all_df)}")
filtered_num_all_df.head()

Total number of financial facts extracted: 5810260


,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote,quarter
0,0001025378-24-000037,AllocatedShareBasedCompensationExpense,us-gaap/2023,20211231,4,USD,NaN,NaN,2.488100e+07,NaN,2024q1
1,0001213900-24-027206,AdjustmentsToAdditionalPaidInCapitalShareBased...,us-gaap/2023,20221231,4,USD,NaN,NaN,7.149000e+06,NaN,2024q1
2,0000950170-24-028127,AdjustmentsToAdditionalPaidInCapitalSharebased...,us-gaap/2023,20211231,4,USD,NaN,NaN,5.176000e+06,NaN,2024q1
3,0001628280-24-006850,LongTermDebtNoncurrent,us-gaap/2023,20221231,0,USD,DebtInstrument=A415NotesDue2043;,NaN,1.239000e+09,NaN,2024q1
4,0001174947-24-000361,PolicyholderBenefitsAndClaimsIncurredNet,us-gaap/2023,20221231,4,USD,BusinessSegments=NonStandardAuto;,NaN,3.940000e+07,NaN,2024q1


In [10]:
# Tags
interesting_tags = [
    "Revenues",
    "NetIncomeLoss",
    "Assets",
    "Liabilities",
    "CashAndCashEquivalentsAtCarryingValue"
]

In [11]:
# Filter the financial facts by these tags
facts_df = filtered_num_all_df[ filtered_num_all_df["tag"].isin(interesting_tags) ].copy()


In [12]:
# Merge with annual_filings_all to get the company name (on 'adsh')
facts_df = facts_df.merge(
    annual_filings_all[["adsh", "name"]].drop_duplicates("adsh"),
    on="adsh",
    how="left"
)

In [13]:
# Remove duplicates (optional, but safe)
facts_df = facts_df.drop_duplicates(subset=["adsh", "tag", "ddate", "value"])


In [14]:
# Create a natural-language sentence
def to_sentence(row):
    company = row["name"]
    tag = row["tag"]
    value = row["value"]
    period = row["ddate"]
    
    # Basic tag → English phrase mapping
    tag_map = {
        "Revenues": "revenue",
        "NetIncomeLoss": "net income",
        "Assets": "total assets",
        "Liabilities": "total liabilities",
        "CashAndCashEquivalentsAtCarryingValue": "cash and cash equivalents"
    }
    
    phrase = tag_map.get(tag, tag)
    
    return f"{company} reported {phrase} of {value} in {period}."

facts_df["sentence"] = facts_df.apply(to_sentence, axis=1)

In [15]:
# Quick check
facts_df[["name","tag","value","ddate","sentence"]].head()

,name,tag,value,ddate,sentence
0,MR. COOPER GROUP INC.,CashAndCashEquivalentsAtCarryingValue,5.710000e+08,20231231,MR. COOPER GROUP INC. reported cash and cash e...
1,PITNEY BOWES INC /DE/,Assets,4.272185e+09,20231231,PITNEY BOWES INC /DE/ reported total assets of...
2,LENDINGCLUB CORP,Liabilities,6.193000e+06,20231231,LENDINGCLUB CORP reported total liabilities of...
3,SNAP-ON INC,Revenues,0.000000e+00,20231231,SNAP-ON INC reported revenue of 0.0 in 20231231.
4,HAWAIIAN ELECTRIC INDUSTRIES INC,NetIncomeLoss,-4.480900e+07,20231231,HAWAIIAN ELECTRIC INDUSTRIES INC reported net ...


In [18]:
import os

processed_dir = os.path.join(ROOT_DIR, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

processed_path = os.path.join(processed_dir, "financial_sentences1.csv")
facts_df.to_csv(processed_path, index=False)

print(f"Saved {len(facts_df)} sentences to {processed_path}")

Saved 221465 sentences to ../data\processed\financial_sentences1.csv


In [19]:
import json

qa_pairs = []

for _, row in facts_df.iterrows():
    company = row["name"]
    tag = row["tag"]
    period = row["ddate"]
    value = row["value"]
    
    # Tag -> phrase mapping (same as before, but for the question)
    tag_map = {
        "Revenues": "revenue",
        "NetIncomeLoss": "net income",
        "Assets": "total assets",
        "Liabilities": "total liabilities",
        "CashAndCashEquivalentsAtCarryingValue": "cash and cash equivalents"
    }
    
    phrase = tag_map.get(tag, tag)
    
    # Build question
    question = f"What was {company}'s {phrase} in {period}?"
    # Build answer
    answer = f"{company} reported {phrase} of {value} in {period}."
    
    qa_pairs.append({"question": question, "answer": answer})

print(f"Total Q/A pairs generated: {len(qa_pairs)}")

# Save
qa_path = os.path.join(ROOT_DIR, "data", "qa_pairs.json")
os.makedirs(os.path.dirname(qa_path), exist_ok=True)

with open(qa_path, "w") as f:
    json.dump(qa_pairs, f, indent=2)

print(f"Q/A pairs saved to: {qa_path}")


Total Q/A pairs generated: 221465
Q/A pairs saved to: ../data\qa_pairs.json


In [20]:
import random

# Sample 50 random pairs
sampled_qa_pairs = random.sample(qa_pairs, 50)

# Save the sampled pairs to a separate file
sampled_path = os.path.join(ROOT_DIR, "data", "qa_pairs_50.json")
with open(sampled_path, "w") as f:
    json.dump(sampled_qa_pairs, f, indent=2)

print(f"Saved a random sample of 50 Q/A pairs to: {sampled_path}")



Saved a random sample of 50 Q/A pairs to: ../data\qa_pairs_50.json


In [21]:
import random

# Sample 50 random pairs
sampled_qa_pairs = random.sample(qa_pairs, 100)

# Save the sampled pairs to a separate file
sampled_path = os.path.join(ROOT_DIR, "data", "qa_pairs_100.json")
with open(sampled_path, "w") as f:
    json.dump(sampled_qa_pairs, f, indent=2)

print(f"Saved a random sample of 100 Q/A pairs to: {sampled_path}")



Saved a random sample of 100 Q/A pairs to: ../data\qa_pairs_100.json
